<a href="https://colab.research.google.com/github/MeatHub/Attention-Is-All-You-Need-Review/blob/main/transformer_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install datasets transformers

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from torch.utils.data import DataLoader, TensorDataset
from datasets import load_dataset
from transformers import AutoTokenizer
from dataclasses import dataclass
from tqdm.auto import tqdm

In [ ]:
dataset = load_dataset('imdb')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=256)

train_data = dataset['train'].shuffle(seed=42).select(range(2000))
test_data = dataset['test'].shuffle(seed=42).select(range(500))

train_tokenized = train_data.map(tokenize, batched=True)
test_tokenized = test_data.map(tokenize, batched=True)

def make_loader(data, batch_size=16):
    ids = torch.tensor(data['input_ids'])
    labels = torch.tensor(data['label'])
    return DataLoader(TensorDataset(ids, labels), batch_size=batch_size, shuffle=True)

train_loader = make_loader(train_tokenized)
test_loader = make_loader(test_tokenized)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=256, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        # div_term = 1 / 10000^(2i/d_model)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        # PE(pos, 2i) = sin(pos / 10000^(2i/d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        # PE(pos, 2i+1) = cos(pos / 10000^(2i/d_model))
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        # x = x + PE: Embedding 벡터에 위치 정보 합산
        return self.dropout(x + self.pe[:, :x.size(1)])

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super().__init__()
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        self.fc = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, q, k, v):
        batch_size = q.size(0)

        # Linear projection: Q, K, V 생성 (Query, Key, Value)
        # Split into heads: (batch, seq_len, num_heads, d_k) -> (batch, num_heads, seq_len, d_k)
        q = self.w_q(q).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        k = self.w_k(k).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        v = self.w_v(v).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)

        # Scaled Dot-Product Attention: Attention(Q, K, V) = softmax(QK^T / sqrt(d_k))V
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.d_k)
        attn = torch.softmax(scores, dim=-1)

        # MultiHead(Q, K, V) = Concat(head_1, ..., head_h)W^O
        context = torch.matmul(self.dropout(attn), v).transpose(1, 2).reshape(batch_size, -1, self.num_heads * self.d_k)
        return self.fc(context)

In [ ]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super().__init__()
        self.mha = MultiHeadAttention(d_model, num_heads, dropout)
        self.norm1 = nn.LayerNorm(d_model)

        # FFN(x) = max(0, xW1 + b1)W2 + b2
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_model * 4),
            nn.ReLU(),
            nn.Linear(d_model * 4, d_model)
        )
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Sublayer 1: Residual Connection & Layer Normalization -> LayerNorm(x + Sublayer(x))
        attn_out = self.mha(x, x, x)
        x = self.norm1(x + self.dropout(attn_out))

        # Sublayer 2: Position-wise Feed-Forward Network & LayerNorm
        ffn_out = self.ffn(x)
        x = self.norm2(x + self.dropout(ffn_out))
        return x

In [ ]:
class TransformerSentimentModel(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, num_layers=2, num_classes=2, dropout=0.1):
        super().__init__()
        # Input Embedding: 문자를 d_model 차원의 벡터로 변환
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model, dropout=dropout)

        # N x Encoder Layers: 논문의 Nx 구조 (중첩된 인코더 층)
        self.layers = nn.ModuleList([TransformerEncoderLayer(d_model, num_heads, dropout) for _ in range(num_layers)])

        # Final Linear Layer: 분류를 위한 출력층
        self.classifier = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoding(x)
        for layer in self.layers:
            x = layer(x)

        # Global Average Pooling: (batch, seq_len, d_model) -> (batch, d_model)
        x = x.mean(dim=1)
        return self.classifier(x)

In [ ]:
class Trainer:
    def __init__(self, model, config, train_loader, test_loader):
        self.model = model.to(config.device)
        self.config = config
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=config.lr)
        self.criterion = nn.CrossEntropyLoss()

    def train(self):
        for epoch in range(self.config.epochs):
            self.model.train()
            total_loss = 0
            for batch in tqdm(self.train_loader, desc=f"Epoch {epoch+1}"):
                ids, labels = [b.to(self.config.device) for b in batch]
                self.optimizer.zero_grad()
                loss = self.criterion(self.model(ids), labels)
                loss.backward()
                self.optimizer.step()
                total_loss += loss.item()

            acc = self.evaluate()
            print(f"Loss: {total_loss/len(self.train_loader):.4f} | Test Acc: {acc:.2f}%")

    def evaluate(self):
        self.model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for batch in self.test_loader:
                ids, labels = [b.to(self.config.device) for b in batch]
                out = self.model(ids)
                correct += (out.argmax(1) == labels).sum().item()
                total += labels.size(0)
        return 100 * correct / total

def predict_sentiment(text, model, tokenizer, device):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors="pt", padding='max_length',
                          truncation=True, max_length=256).to(device)
        output = model(inputs['input_ids'])
        prob = torch.softmax(output, dim=-1)
        pred = output.argmax(1).item()
        label = "긍정 😊" if pred == 1 else "부정 😡"
        print(f"리뷰: {text}\n결과: {label} ({prob[0][pred].item()*100:.2f}%)")
        print("-" * 50)

In [ ]:
@dataclass
class Config:
    vocab_size: int = tokenizer.vocab_size
    d_model: int = 128
    num_heads: int = 8
    epochs: int = 12
    lr: float = 1e-4
    device: torch.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = Config()
model = TransformerSentimentModel(config.vocab_size, config.d_model, config.num_heads)

In [ ]:
trainer = Trainer(model, config, train_loader, test_loader)
trainer.train()

Epoch 1:   0%|          | 0/125 [00:00<?, ?it/s]

Loss: 0.7009 | Test Acc: 53.80%


Epoch 2:   0%|          | 0/125 [00:00<?, ?it/s]

Loss: 0.6909 | Test Acc: 53.60%


Epoch 3:   0%|          | 0/125 [00:00<?, ?it/s]

...

Loss: 0.4293 | Test Acc: 72.60%


Epoch 11:   0%|          | 0/125 [00:00<?, ?it/s]

Loss: 0.4105 | Test Acc: 74.20%


Epoch 12:   0%|          | 0/125 [00:00<?, ?it/s]

Loss: 0.3814 | Test Acc: 74.20%


In [ ]:
test_texts = [
    "This movie was a masterpiece. The depth of the characters was incredible.",
    "I hated this film. It was way too long and very boring.",
    "It was okay, but the ending was a bit disappointing."
]

for text in test_texts:
    predict_sentiment(text, model, tokenizer, config.device)

리뷰: This movie was a masterpiece. The depth of the characters was incredible.
결과: 긍정 😊 (96.33%)
--------------------------------------------------
리뷰: I hated this film. It was way too long and very boring.
결과: 부정 😡 (75.41%)
--------------------------------------------------
리뷰: It was okay, but the ending was a bit disappointing.
결과: 긍정 😊 (83.33%)
--------------------------------------------------
